In [5]:
import pandas as pd
import re
import os

ModuleNotFoundError: No module named 'pandas'

# Passo 1: Carregar os dados 🧠


### Define o caminho relativo para o arquivo de dados
### '..' significa 'voltar uma pasta' (de /notebooks para /evasao-zero)

In [ ]:
caminho_dados_originais = os.path.join('..', 'data', 'planilha_original.csv')

try:
    df = pd.read_csv(caminho_dados_originais)
    print("Arquivo de dados original carregado com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em '{caminho_dados_originais}'.")
    print("Verifique se o arquivo 'planilha_original.csv' está na pasta 'data'.")

# Passo 2: Limpeza e padronização 🚨

In [ ]:
# Função para limpar e padronizar nomes de colunas
def limpar_nomes_colunas(df):
    colunas_limpas = []
    for col in df.columns:
        # Remove espaços no início/fim, converte para minúsculas
        col_limpa = col.strip().lower()
        # Substitui espaços e caracteres especiais por underscore
        col_limpa = re.sub(r'\s+', '_', col_limpa)
        col_limpa = re.sub(r'[^a-z0-9_]', '', col_limpa)
        colunas_limpas.append(col_limpa)
    df.columns = colunas_limpas
    return df

df_limpo = limpar_nomes_colunas(df.copy())

# Renomear colunas específicas para nomes mais curtos e claros
mapeamento_nomes = {
    'qual_seu_curso': 'curso',
    'em_que_perodo_voc_est': 'periodo',
    'qual_sua_idade': 'idade',
    'com_qual_gnero_voc_se_identifica': 'genero',
    'voc_se_considera_uma_pessoa': 'cor_raca',
    'qual_a_renda_mensal_total_da_sua_famlia_some_os_salrios_de_todos_que_contribuem_com_as_despesas_da_casa': 'renda_familiar',
    'voc_trabalha_atualmente': 'trabalha',
    'voc_j_pensou_em_desistir_ou_trancar_o_curso': 'pensou_desistir',
    'se_respondeu_sim_na_anterior_quais_foram_os_principais_motivos_que_te_levaram_a_pensar_em_desistir': 'motivos_desistencia',
    'na_sua_opinio_o_que_a_instituio_poderia_fazer_para_ajudar_os_alunos_a_no_desistirem_dos_seus_cursos': 'feedback_melhoria'
}
df_limpo.rename(columns=mapeamento_nomes, inplace=True)

# Padronização da coluna 'genero'
df_limpo['genero'] = df_limpo['genero'].replace({
    'Homem Cisgênero': 'Homem',
    'Homem/Cisgênero': 'Homem',
    'Mulher Cisgênero': 'Mulher'
})

# Limpeza e conversão da coluna 'periodo' para numérico
df_limpo['periodo'] = df_limpo['periodo'].str.extract(r'(\d+)').astype(int)

# Criação de uma nova coluna categórica simplificada para 'pensou_desistir'
def categorizar_desistencia(resposta):
    if isinstance(resposta, str):
        if 'Não, nunca' in resposta:
            return 'Não'
        if 'Sim, várias vezes' in resposta or 'Sim, uma vez' in resposta:
            return 'Sim'
    return 'Talvez' # Para 'Talvez' ou outros valores nulos/diferentes

df_limpo['risco_evasao_declarado'] = df_limpo['pensou_desistir'].apply(categorizar_desistencia)

# Passo 3: Salvar os dados limpos 🎲

In [ ]:
caminho_dados_limpos = os.path.join('..', 'data', 'dados_limpos.csv')
df_limpo.to_csv(caminho_dados_limpos, index=False)

print(f"\nArquivo limpo e padronizado foi salvo com sucesso em: '{caminho_dados_limpos}'")

# --- Passo 4: Verificar o Resultado ---
print("\nAmostra dos dados após a limpeza:")
print(df_limpo[['curso', 'periodo', 'genero', 'risco_evasao_declarado']].head())